# 🛒 Grocery Sales Analysis Framework

This notebook implements a comprehensive data analysis pipeline for grocery sales data. It covers data loading, preprocessing, metric calculation, and interactive visualization.

## 1. Setup & Installation

In [1]:
!pip install pandas plotly

## 2. File Upload

Please upload the following files: `annex1.csv`, `annex2.csv`, `annex3.csv`, `annex4.csv`.

In [2]:
import os
from google.colab import files

# Check if files exist, else upload
required_files = ['annex1.csv', 'annex2.csv', 'annex3.csv', 'annex4.csv']
missing_files = [f for f in required_files if not os.path.exists(f)]

if missing_files:
    print(f'Please upload: {missing_files}')
    uploaded = files.upload()
else:
    print('All required files detected.')

Please upload: ['annex1.csv', 'annex2.csv', 'annex3.csv', 'annex4.csv']


Saving annex2.csv to annex2.csv
Saving annex3.csv to annex3.csv
Saving annex4.csv to annex4.csv
Saving annex1.csv to annex1.csv


## 3. Data Processing Pipeline

In [3]:
import pandas as pd
import plotly.express as px

def load_and_process_data(base_path='.'):
    print('Loading data...')
    try:
        items_df = pd.read_csv(os.path.join(base_path, 'annex1.csv'))
        transactions_df = pd.read_csv(os.path.join(base_path, 'annex2.csv'))
        wholesale_df = pd.read_csv(os.path.join(base_path, 'annex3.csv'))
        loss_df = pd.read_csv(os.path.join(base_path, 'annex4.csv'))
    except FileNotFoundError as e:
        print(f'Error: {e}')
        return None

    # 1. Standardization
    for df in [items_df, transactions_df, wholesale_df, loss_df]:
        df.columns = df.columns.str.strip()

    # 2. DateTime Conversion
    transactions_df['DateTime'] = pd.to_datetime(
        transactions_df['Date'] + ' ' + transactions_df['Time'],
        format='mixed'
    )
    transactions_df['Date'] = pd.to_datetime(transactions_df['Date'])
    wholesale_df['Date'] = pd.to_datetime(wholesale_df['Date'])

    # 3. Merging
    # Merge Item Info
    merged_df = transactions_df.merge(items_df, on='Item Code', how='left')

    # Merge Wholesale Price (Time-series)
    merged_df = merged_df.merge(wholesale_df, on=['Date', 'Item Code'], how='left')

    # Merge Loss Rate
    loss_df = loss_df[['Item Code', 'Loss Rate (%)']]
    merged_df = merged_df.merge(loss_df, on='Item Code', how='left')
    merged_df['Loss Rate (%)'] = merged_df['Loss Rate (%)'].fillna(0)

    # 4. Filling Missing Prices
    merged_df = merged_df.sort_values(by=['Item Code', 'DateTime'])
    merged_df['Wholesale Price (RMB/kg)'] = merged_df.groupby('Item Code')['Wholesale Price (RMB/kg)'].ffill()
    merged_df['Wholesale Price (RMB/kg)'] = merged_df.groupby('Item Code')['Wholesale Price (RMB/kg)'].bfill()

    # 5. Metrics Calculation
    merged_df['Revenue'] = merged_df['Quantity Sold (kilo)'] * merged_df['Unit Selling Price (RMB/kg)']
    merged_df['Base Cost'] = merged_df['Quantity Sold (kilo)'] * merged_df['Wholesale Price (RMB/kg)']
    # Effective Cost = Cost / (1 - Loss%)
    merged_df['Effective Cost'] = merged_df['Base Cost'] / (1 - (merged_df['Loss Rate (%)'] / 100))
    merged_df['Net Profit'] = merged_df['Revenue'] - merged_df['Effective Cost']

    print('Data processed successfully.')
    return merged_df

df = load_and_process_data()

Loading data...
Data processed successfully.


## 4. Key Performance Indicators (KPIs)

In [4]:
if df is not None:
    total_revenue = df['Revenue'].sum()
    total_profit = df['Net Profit'].sum()
    total_qty = df['Quantity Sold (kilo)'].sum()
    profit_margin = (total_profit / total_revenue * 100) if total_revenue > 0 else 0

    print(f'Total Revenue: ¥{total_revenue:,.2f}')
    print(f'Total Profit:  ¥{total_profit:,.2f}')
    print(f'Profit Margin: {profit_margin:.2f}%')
    print(f'Total Quantity: {total_qty:,.2f} kg')

Total Revenue: ¥3,369,766.48
Total Profit:  ¥1,003,953.18
Profit Margin: 29.79%
Total Quantity: 470,975.92 kg


## 5. Visualizations

### Daily Sales Trends

In [5]:
daily_df = df.groupby('Date').agg({
    'Revenue': 'sum',
    'Net Profit': 'sum'
}).reset_index()

fig = px.line(daily_df, x='Date', y=['Revenue', 'Net Profit'],
              title='Daily Revenue and Profit Trends',
              color_discrete_map={'Revenue': '#29b5e8', 'Net Profit': '#e15759'})
fig.show()

### Category Analysis

In [6]:
cat_metrics = df.groupby('Category Name').agg({
    'Revenue': 'sum',
    'Net Profit': 'sum'
}).reset_index().sort_values(by='Revenue', ascending=False)

fig_bar = px.bar(cat_metrics, x='Category Name', y='Revenue',
                 title='Revenue by Category', color='Revenue',
                 color_continuous_scale='Blues')
fig_bar.show()

fig_pie = px.pie(cat_metrics, names='Category Name', values='Net Profit',
                 title='Profit Share by Category')
fig_pie.show()

### Top/Bottom Items

In [7]:
# Top 10 by Profit
top_items = df.groupby('Item Name')['Net Profit'].sum().reset_index()
top_items = top_items.sort_values(by='Net Profit', ascending=False).head(10)

fig_item = px.bar(top_items, x='Net Profit', y='Item Name', orientation='h',
                  title='Top 10 Items by Net Profit', text_auto='.2s')
fig_item.update_layout(yaxis={'categoryorder':'total ascending'})
fig_item.show()